In [0]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext

from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col

import pandas as pd

# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

triplets_file = 'https://static.turi.com/datasets/millionsong/10000.txt'
song_df_1 = pd.read_table(triplets_file,header=None)
song_df_1.columns = ['user_id', 'song_id', 'listen_count']
song_df_1.to_csv('tripletas.csv')



     |████████████████████████████████| 217.8MB 64kB/s 
     |████████████████████████████████| 204kB 44.5MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=61db74e86d14b6d4f7520db3d7ddea8a3f679e5b9cd0c6c4c207bc3189110307
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark
openjdk-8-jdk-headless is already the newest version (8u242-b08-0ubuntu3~18.04).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [0]:
from google.colab import drive
drive.mount('/content/drive')



# User-User

In [0]:
from pyspark.sql.types import FloatType, ArrayType, StringType
triplets_file = 'tripletas.csv'
song_df = spark.read.csv(triplets_file, header='true')
user_mean = song_df.groupBy('user_id').agg(mean('listen_count').alias("media"))
song_mean = song_df.join(user_mean,on='user_id', how='inner')
song_mean = song_mean.withColumn('count_mean',col('listen_count')/col('media'))
only_jams=song_mean.filter(song_mean.listen_count>song_mean.media).select(['user_id','song_id'])



def jaccard_similarity(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union


def news(s1,s2):
    return list(set(s2).difference(set(s1)))


user_set_jams = only_jams.groupby("user_id").agg(collect_set("song_id").alias('Set_Songs'))#.limit(100) ### para cada usuario el set de lo que le gusta
user_set_jams_2 = user_set_jams.withColumnRenamed('user_id','user_id_2')
user_set_jams_2 = user_set_jams_2.withColumnRenamed('Set_Songs','Set_Songs_2')


### Usando pre filtro Pre filtro

only_jams2 = only_jams.withColumnRenamed('user_id','user_id_2')

pre_double_jams = only_jams.join(only_jams2,on='song_id')

pre_double_jams = pre_double_jams.filter( pre_double_jams.user_id != pre_double_jams.user_id_2).select('user_id','user_id_2')


pre_double_jams = pre_double_jams.groupBy('user_id','user_id_2').agg(count('*').alias('number_common'))

pre_double_jams = pre_double_jams.filter(pre_double_jams.number_common>1)

pre_double_jams = pre_double_jams.select('user_id','user_id_2')

pre_double_jams = pre_double_jams.dropDuplicates()

double_jams = pre_double_jams.join(user_set_jams,on='user_id').join(user_set_jams_2,on='user_id_2') ### AQui tenemos un dataframe de 2 usuarios con sus listas de jams



### Sin pre filtro
#double_jams = user_set_jams.crossJoin(user_set_jams_2)
#double_jams = double_jams.filter(double_jams.user_id!=double_jams.user_id_2)




news_udf = udf(lambda y: news(y[0],y[1]), ArrayType(StringType()))

jack = udf(lambda x: jaccard_similarity(x[0],x[1]), FloatType())

user_jack=double_jams.withColumn('Jackard', jack(array('Set_Songs', 'Set_Songs_2'))).select('user_id','user_id_2','Jackard') ### AQUI calculamos que tan aprecidos son 2 usuarios

user_jack=user_jack.filter(user_jack.Jackard>0)

window = Window.partitionBy(user_jack['user_id']).orderBy(user_jack['Jackard'].desc())

same_user = user_jack.select('*', rank().over(window).alias('rank')).\
                  filter(col('rank') <= 10).select('user_id','user_id_2','Jackard') #### Para cada usuario voy a ver cuales son sus 10 más aprecidos

#same_user.toPandas().to_csv('drive/My Drive/Ms Ciencia de datos/user-sim.csv')

same_user = same_user.withColumnRenamed('user_id','user_id_new')
same_user = same_user.withColumnRenamed('user_id_2','user_id')


user_jams = only_jams.groupby("user_id").agg(collect_set("song_id").alias('Set_songs')) ### Para cada usuario veo cual es su lista de jams


resultado = only_jams.join(same_user, on = 'user_id').groupby(["user_id_new","song_id"]).\
          agg(sum("Jackard").alias('score_recomendacion')).\
          sort(desc("score_recomendacion")).\
          withColumnRenamed('user_id_new','user_id').\
          groupby("user_id").agg(collect_set("song_id").alias('Set_recomendaciones')).\
          join(user_jams, on = 'user_id').\
          withColumn('recomendacion', news_udf(array('Set_Songs', 'Set_recomendaciones'))).\
          select('user_id','recomendacion') ### Para cada usuario se le dan las recomendaciones con base en los 10 usuarios alike segun jaccrd
          ### Importante ver que se suman todos los jams de los alikes y al final se pondera cada voto segun que tan parecidos son
          ### Finalmente el resultado es ordenado y solo se recomienda lo que no está en el set de jams de la persona


%time resultado.show()



# Song-Song

In [0]:
triplets_file = 'tripletas.csv'
song_df = spark.read.csv(triplets_file, header='true')
user_mean = song_df.groupBy('user_id').agg(mean('listen_count').alias("media"))
song_mean = song_df.join(user_mean,on='user_id', how='inner')
song_mean = song_mean.withColumn('count_mean',col('listen_count')/col('media'))

only_jams=song_mean.filter(song_mean.listen_count>song_mean.media*0.8).select(['user_id','song_id'])

jams_set_users = only_jams.groupby("song_id").agg(collect_set("user_id").alias('Set_users'))# Para cada cancion tenemos el grupo de usuarios que le gustan

jams_set_users_2 = jams_set_users.withColumnRenamed('song_id','song_id_2')

jams_set_users_2 = jams_set_users_2.withColumnRenamed('Set_users','Set_users_2')

def jaccard_similarity(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

def news(s1,s2):
    return list(set(s2).difference(set(s1)))

jack = udf(lambda x: jaccard_similarity(x[0],x[1]), FloatType())

news_udf = udf(lambda y: news(y[0],y[1]), ArrayType(StringType()))

### El siguiente bloque es para solo calcular jaccard entre las canciones que esten el al menos el jamlist de 2 usuarios
only_jams2 = only_jams.withColumnRenamed('song_id','song_id_2')

double_jams = only_jams.join(only_jams2,on='user_id')

double_jams = double_jams.filter(double_jams.song_id!=double_jams.song_id_2)

initial_double_songs = double_jams.groupBy('song_id','song_id_2').agg(count('*').alias('number_common'))

initial_double_songs = initial_double_songs.filter(initial_double_songs.number_common>1)

initial_double_songs = initial_double_songs.select('song_id','song_id_2')

#####


double_users = initial_double_songs.join(jams_set_users, on = 'song_id').join(jams_set_users_2, on = 'song_id_2')## en el mismo dataframe ya tenemos la lista de usuarios a los que le gusta de cada par de cnaciones que nos interesan

double_users = double_users.filter(double_users.song_id!=double_users.song_id_2)

initial_rec=double_users.withColumn('Jackard', jack(array('Set_users', 'Set_users_2'))) ### Calculo jackard para las dos canciones

initial_rec = initial_rec.filter(initial_rec.Jackard>0) ### Solo me interesan los que le gustan

window = Window.partitionBy(initial_rec['song_id']).orderBy(initial_rec['Jackard'].desc())

same_song = initial_rec.select('*', rank().over(window).alias('rank')).\
                  filter(col('rank') <= 10).select('song_id','song_id_2','Jackard') ### Para cada cancion me quedo con las 10 más similares a ella

#same_song.toPandas().to_csv('drive/My Drive/Ms Ciencia de datos/song-sim.csv')

#same_song.toPandas.to_csv('song_song_similarity.csv') # Esto es lo más lento, por eso mejor guardarlo

user_jams = only_jams.groupby("user_id").agg(collect_set("song_id").alias('Set_songs')) ### Para cada usuario veo cual es su lista de jams

resultado = only_jams.join(same_song,on='song_id').groupby(["user_id","song_id_2"]).\
          agg(sum("Jackard").\
          alias('score_recomendacion')).\
          sort(desc("score_recomendacion")).\
          groupby("user_id").agg(collect_set("song_id_2").alias('Set_recomendaciones')).\
          join(user_jams, on = 'user_id').\
          withColumn('recomendacion', news_udf(array('Set_Songs', 'Set_recomendaciones'))).\
          select('user_id','recomendacion') ### segun las canciones que le gustan a cada persona, le recomiendo todo lo que "es igual" segun same songs y que no está en su jams list

%time resultado.show()
